In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
from scipy import stats

In [ ]:
#Loading dataset

In [ ]:
# df=pd.read_parquet('test_11_10.parquet')
df = pd.read_csv('Energy_meter_data.csv')
df['Time'] = pd.to_datetime(df['Time'])
df

In [ ]:
check = ['Phase1_I', 'Phase2_I', 'Phase3_I', 'Phase1_V_RY', 'Phase2_V_YB', 'Phase3_V_BR']
df['Status'] = (df[check] >= 10).all(axis=1)

In [ ]:
df.rename(columns={'Phase1_W':'Phase1_kW','Phase2_W':'Phase_kW', 'Phase3_W':'Phase3_kW', 'W':'kW'})
list1 = ['Phase1_W', 'Phase2_W', 'Phase3_W', 'W','VA']
list2 =['Phase1_W','Phase2_W','Phase3_W','W','PF']
list_negative = ['Phase1_pf','Phase2_pf','Phase3_pf','Phase1_W','Phase2_W','Phase3_W','PF']
df[list1] = df[list1].apply(lambda x: x / 1000)
df[list2] = df[list2].apply(lambda x: x *-1)
#df[list_pf] = df[list_pf].apply(lambda x: x.where(x < 0, -1*x))
#df[list_negative] = df[list_negative].apply(lambda x: x.where(x >= 0, -x))
df[list_negative] = df[list_negative].apply(lambda x: x.where(x >= 0.000000, abs(x)))

In [ ]:
def calculate_voltage_imbalance(row):
    ##### Pump considered off when Phase current and volatge = 0
    currents_volt = [row['Phase1_I'], row['Phase2_I'], row['Phase3_I'],row['Phase1_V_RY'], row['Phase2_V_YB'], row['Phase3_V_BR']]
    if all(i != 0 for i in currents_volt): 
        voltages = [row['Phase1_V_RY'], row['Phase2_V_YB'], row['Phase3_V_BR']]
  #  if all(v != 0 for v in voltages):  # not applicable hen the Pump is off
        avg_voltage = sum(voltages) / len(voltages)
        max_deviation = max(voltages) - avg_voltage
        voltage_imbalance = (max_deviation / avg_voltage) * 100
        return voltage_imbalance
    else:
        return None


In [ ]:
def calculate_current_imbalance(row):
    currents = [row['Phase1_I'], row['Phase2_I'], row['Phase3_I']]
    if all(i != 0 for i in currents):  # doesnot apply when the pump is off
        avg_current = sum(currents) / len(currents)
        max_current_deviation = max(currents)-avg_current
        current_imbalance = ((max_current_deviation) / avg_current) * 100
        return current_imbalance
    else:
        return None

In [ ]:
df['Voltage_Imbalance_%'] = df.apply(calculate_voltage_imbalance, axis=1)
df['Current_Imbalance_%'] = df.apply(calculate_current_imbalance, axis=1)
df

In [ ]:
df.set_index('Time', inplace=True)

In [ ]:
df.dtypes

In [ ]:
df[['MeterId','W']]

In [ ]:
def meter_id_correction(row):
    if row['MeterId'] == 1:
        return row['Phase1_W']+row['Phase2_W']+row['Phase3_W']
    else:
        return row['W']

In [ ]:
df['W'] = df.apply(meter_id_correction,axis=1)

In [ ]:
df[['MeterId','W']]

In [ ]:
head= 39 ## m
flow_rate = 932/3600 ## m3/s
density = 1000 ### kg/m3
g = 9.8 ## m/s2

P_out= (density*g*flow_rate*head)/1000    # power in kW

In [ ]:
df['Efficiency'] = (P_out/df['W'])*100
df

In [ ]:
df = df[df['Status']!=False]

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df[df['Voltage_Imbalance_%'].isna()]

In [ ]:
df.iloc[:,:19]

In [ ]:
df.columns

In [ ]:
df[df.index>='2024-10-13']

In [ ]:
meter_dfs = {}
for i in df['MeterId'].unique():
    meter_dfs[f'Pump_{i}'] = df[df['MeterId'] == i][[#'Time',
         'Phase1_W', 'Phase1_I', 'Phase1_V_RY', 'Phase1_pf',
        'Phase2_W', 'Phase2_I', 'Phase2_V_YB', 'Phase2_pf',
        'Phase3_W', 'Phase3_I', 'Phase3_V_BR', 'Phase3_pf',
        'W', 'VA', 'PF', 'KWh_Im', 'KWh_Ex', 'KVAh', 'Lh','Status','Voltage_Imbalance_%','Current_Imbalance_%',
        'Efficiency'
    ]]

In [ ]:
pump_1_df = meter_dfs['Pump_1']
pump_2_df = meter_dfs['Pump_2']
pump_3_df = meter_dfs['Pump_3']
pump_5_df = meter_dfs['Pump_5']
pump_6_df = meter_dfs['Pump_6']

In [ ]:
df1= pump_1_df[pump_1_df['Status']!=False]
df2= pump_2_df[pump_2_df['Status']!=False]
df3= pump_3_df[pump_3_df['Status']!=False]
df5= pump_5_df[pump_5_df['Status']!=False]
df6= pump_6_df[pump_6_df['Status']!=False]

In [ ]:
df1.columns

In [ ]:
# df1['Efficiency'] = (P_out/df1['W'])*100
# df2['Efficiency'] = (P_out/df2['W'])*100
# df3['Efficiency'] = (P_out/df3['W'])*100
# df5['Efficiency'] = (P_out/df5['W'])*100
# df6['Efficiency'] = (P_out/df6['W'])*100

In [ ]:
fig = go.Figure()
# dfs = [df1,df1,df1,df1,df1]
# pump = ''
# for i in dfs:
#     if i == df1:
#         pump = 'Pump 1'
#     elif i == df1:
#         pump = 'Pump 2'
#     elif i == df1:
#         pump = 'Pump 3'
#     elif i == df1:
#         pump = 'Pump 5'
#     else:
#         pump = 'Pump 6'
# i = df1
fig.add_trace(go.Scatter(
    x=df1.index,
    y=df1['Voltage_Imbalance_%'],
    mode='lines+markers',
    name='Voltage Imbalance_%',
    line=dict(color='blue'),
    marker=dict(size=5)
))

fig.add_trace(go.Scatter(
    x=df1.index,
    y=df1['Current_Imbalance_%'],
    mode='lines+markers',
    name='Current Imbalance_%',
    line=dict(color='red'),
    marker=dict(size=5)
))

fig.update_layout(
    title=f'Variation of Current and Voltage Imbalances for Pump 1 Over Time',
    xaxis_title='Time',
    yaxis_title='Imbalance %',
    legend_title='Legend',
    template='plotly_white'
)
fig.update_yaxes(
    range=[0, 12], 
    dtick=1         
)


fig.show()

In [ ]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df5.index,
#     y=df5['Phase1_V_RY'],
#     mode='lines+markers',
#     name='Voltage RY',
#     line=dict(color='red'),
#     marker=dict(size=5)
# ))


# # fig.add_trace(go.Scatter(
# #     x=df5['Time'],
# #     y=df5['Phase2_V_YB'],
# #     mode='lines+markers',
# #     name='Current Imbalance',
# #     line=dict(color='yellow'),
# #     marker=dict(size=5)
# # ))
# # fig.add_trace(go.Scatter(
# #     x=df5['Time'],
# #     y=df5['Phase3_V_BR'],
# #     mode='lines+markers',
# #     name='Current Imbalance',
# #     line=dict(color='blue'),
# #     marker=dict(size=5)
# # ))


# fig.update_layout(
#     title='Variation of Current and Voltage Imbalances Over Time',
#     xaxis_title='Time',
#     yaxis_title='Imbalance',
#     legend_title='Legend',
#     template='plotly_white'
# )

# fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df1.index,
    y=df1['PF'],
    mode='lines+markers',
#     line_shape='spline',
    name='Voltage RY',
    line=dict(color='red'),
    marker=dict(size=5)
))
fig.update_layout(
    title='Variation of PF for Pump 1 Over Time',
    xaxis_title='Time',
    yaxis_title='Power Factor',
    legend_title='Legend',
    template='plotly_white'
)
fig.update_yaxes(
    range=[0.6, 1], 
    dtick=0.02         
)


fig.show()

In [ ]:
# fig = go.Figure()

# # Add a trace for Voltage Imbalance
# fig.add_trace(go.Scatter(
#     x=df6.index,
#     y=df6['W'],
#     mode='lines+markers',
#     name='Voltage RY',
#     line=dict(color='red'),
#     marker=dict(size=5)
# ))
# fig.update_layout(
#     title='Variation of Power of Pump6 Over Time',
#     xaxis_title='Time',
#     yaxis_title='Imbalance',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[120, 150]
# #     dtick=0.02         
# )


# fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df6.index,
    y=df6['Efficiency'],
    mode='lines+markers',
 #   name='Voltage Imbalance_%',
    line=dict(color='blue'),
    marker=dict(size=5)
))


fig.update_layout(
    title='Variation of Efficiency for Pump 6 Over Time',
    xaxis_title='Time',
    yaxis_title='Efficiency %',
    legend_title='Legend',
    template='plotly_white'
)
fig.update_yaxes(
    range=[60, 100],  
    dtick=10         
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df1.index,
    y=df1['W'],
    mode='lines+markers',
    name='Voltage Imbalance_%',
    line=dict(color='blue'),
    marker=dict(size=5)
))


fig.add_shape(
    type='line',
    x0=df1.index.min(),  
    y0=132,              ## Rated power of motor
    x1=df1.index.max(),  
    y1=132,               # Y-value for the reference line
    line=dict(color='green', width=2, dash='dash'),  # Line style
)


fig.update_layout(
    title='Variation of Power for Pump 1 Over Time',
    xaxis_title='Time',
    yaxis_title='Power (kW)',
    legend_title='Legend',
    template='plotly_white'
)
fig.update_yaxes(
    range=[120, 150]
#     dtick=0.02         
)

fig.show()

In [ ]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df1.index,
#     y=df1['W'],
#     mode='lines+markers',
#     name='Voltage Imbalance_%',
#     line=dict(color='blue'),
#     marker=dict(size=5)
# ))


# fig.add_shape(
#     type='line',
#     x0=df1.index.min(),  
#     y0=132,               ## rated power-motor
#     x1=df1.index.max(),  
#     y1=132,               
#     line=dict(color='green', width=2, dash='dash'),  # Line style
# )


# fig.update_layout(
#     title='Power Variation of Pump1 Over Time',
#     xaxis_title='Time',
#     yaxis_title='Imbalance',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[120, 150],  
#     dtick=10         
# )

# fig.show()

In [ ]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df5.index,
#     y=df5['Efficiency'],
#     mode='lines+markers',
#     name='Voltage Imbalance_%',
#     line=dict(color='blue'),
#     marker=dict(size=5)
# ))


# fig.update_layout(
#     title='Efficiency Variation of Pump5',
#     xaxis_title='Time',
#     yaxis_title='Efficiency %',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[60, 80],  
#     dtick=2         
# )

# fig.show()

In [ ]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df3.index,
#     y=df3['Efficiency'],
#     mode='lines+markers',
#     name='Voltage Imbalance_%',
#     line=dict(color='blue'),
#     marker=dict(size=5)
# ))


# fig.update_layout(
#     title='Efficiency Variation of Pump3',
#     xaxis_title='Time',
#     yaxis_title='Efficiency %',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[60, 80],  
#     dtick=10         
# )

# fig.show()

In [ ]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df6.index,
#     y=df6['Efficiency'],
#     mode='lines+markers',
#     name='Voltage Imbalance_%',
#     line=dict(color='blue'),
#     marker=dict(size=5)
# ))


# fig.update_layout(
#     title='Efficiency Variation of Pump6',
#     xaxis_title='Time',
#     yaxis_title='Efficiency %',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[60, 90],  
#     dtick=10         
# )

# fig.show()